# Рекомендация тарифов

На исследование представлены данные о поведении клиентов и выбранные тарифные планы. Необходимо построить модель для задачи классификации, которая выберет подходящий тариф.

Требуется построить модель с максимально большим значением accuracy. Долю правильных ответов должна быть не ниже 0.75. 

In [112]:
# импорт необходимых для эксперимета библиотек
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Чтение файла

In [113]:
# чтение представленное на эксперимент файла
df = pd.read_csv('/datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Файл прочитался корректно.

**Описание данных:**

- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [114]:
# размеры матрицы
df.shape

(3214, 5)

В данных предствлено 3214 объекта, по объектам имеется 4 признака и целевая переменная.

In [115]:
# пропорции целевой переменной
df['is_ultra'].value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

Целевая переменная - тарифный план, представлена в пропорции 0.7 - тариф "Смарт", 0.3 - тариф "Ультра".

Таким образом, целевая переменная несбалансирована.

In [116]:
# общие сведения о данных
df.info()

# сведения о дубликатах
df.duplicated().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


0

Пропущенные значения, явные(полные) дубликаты - отсутсвуют. Данные представлены целочисленными типами и типами с плавающей точкой.

## Разделение выборок на обучающую, валидационную и тестовую

In [117]:
# признаки объектов
X = df.drop(['is_ultra'], axis=1)

# целевая переменная
y = df['is_ultra']

# разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                      test_size=.4, 
                                                      stratify=y,      # целевая переменная несбалансирована,
                                                                       # требуется стратификация
                                                      random_state=42)

# разделение данных на тестовую валидационную выборки
X_test, X_valid, y_test, y_valid = train_test_split(X, y, 
                                                      test_size=.5, 
                                                      stratify=y,      # целевая переменная несбалансирована,
                                                                       # требуется стратификация
                                                      random_state=42)

## Исследование моделей
### Логистическая регрессия

In [118]:
# проверка признаков на коллинеарность
X.corr()

,calls,minutes,messages,mb_used
calls,1.000000,0.982083,0.177385,0.286442
minutes,0.982083,1.000000,0.173110,0.280967
messages,0.177385,0.173110,1.000000,0.195721
mb_used,0.286442,0.280967,0.195721,1.000000


Признак количество звонков сильно коллерирует с продолжительностью разговоров. Для постоения модели логистической регрессии исключим признак количество звонков из выборки.

In [119]:
# инициализация модели
log_reg_model = LogisticRegression(random_state=42)

# обучение модели на обучающей выборке
log_reg_model.fit(X_train.drop(['calls'], axis=1), y_train)

# предсказание целевой переменной по признакам валидационной выборки
log_reg_model_predictions = log_reg_model.predict(X_valid.drop(['calls'], axis=1))

# оценка качества модели показателем метрики - доля правильный предсказаний
print('Mодель:', log_reg_model)
print('Accuracy:', accuracy_score(y_valid, log_reg_model_predictions))

Mодель: LogisticRegression(random_state=42)
Accuracy: 0.7398879900435594


Таким образом, доля верных предстказаний модели логистической регресии - 0,7398.

### Деревья классификации

In [120]:
# подбор гиперпараметра - количество слоев дерева в диапазоне от 1 до 5 слоев 
# по показателю метрики - доля верных предстказаний
best_model = None
best_result = 0.6990                                                          # accuracy логистической регрессии
best_depth = 0
for depth in range(1, 6):
    des_tree_model = DecisionTreeClassifier(max_depth=depth, random_state=42) # инициализация модели
    
    des_tree_model.fit(X_train, y_train)                                      # обучение модели на обучающей выборке
    
    des_tree_model_predictions = des_tree_model.predict(X_valid)              # предсказание целевой переменной 
                                                                              # по признакам валидационной выборки
    
    result = accuracy_score(y_valid, des_tree_model_predictions)              # расчет метрики accuracy
    if result > best_result:                                                  # определение гиперпараметров
        best_model = des_tree_model                                           # лучшей модели
        best_result = result
        best_depth = depth
            
print('Лучшая модель:', best_model)
print('Accuracy:', best_result) 

Лучшая модель: DecisionTreeClassifier(max_depth=5, random_state=42)
Accuracy: 0.7958929682638457


Таким образом, доля верных предсказаний, равная 0,7958 модели дерево решений, достигается при глубине слоев дерева равное 5.

### Случайный лес

In [121]:
# подбор гиперпараметров количество лесов (от 10 до 50 с шагом в 10),
# количество слоев в дереве решений
# по значени метрики качества модели - доля вернопредсказаных ответов
best_model = None
best_result = 0.7972                                                         # accuracy модели дерева решений
best_est = 0
best_depth = 0
for est in range(10, 51, 10):                                                # инициализация модели
    for depth in range (1, 11):                                              # подбор гиперпараметров
        rand_forest_model = RandomForestClassifier(random_state=42,
                                                   n_estimators=est,
                                                   max_depth=depth)
        rand_forest_model.fit(X_train, y_train)                              # обучение модели по обучающей выборке
        rand_forest_model_predictions = rand_forest_model.predict(X_valid)   # предсказание результата по выборке валидации
        result = accuracy_score(y_valid, rand_forest_model_predictions)      # расчет метрики accuracy
        if result > best_result:                                             # определение гиперпараметров
            best_model = rand_forest_model                                   # лучшей модели
            best_result = result
            best_est = est
            best_depth = depth
            
print('Лучшая модель:', best_model)
print('Accuracy:', best_result)

Лучшая модель: RandomForestClassifier(max_depth=9, n_estimators=30, random_state=42)
Accuracy: 0.8238954573739888


Таким образом, доля верных предсказаний, равная 0,8238 модели бэггинга деревьев решений, достигается при глубине слоев деревье равное 9 и количестве деревьев равное 30.

## Проверка на тестовой выборке

In [122]:
# расчет метрики accuracy на тестовой выборке
print('Accuracy:', accuracy_score(y_test, best_model.predict(X_test)))

Accuracy: 0.8780336029869322


Таким образом, доля верных предсказаний модели случайного леса с количеством деревьев равным 30 и количеством слоев в дереве решений равное 9, равна 0,8780, что превышает заявленное на исследование минимальное значение равное 0,7500.

## Проверка модели на адекватность

Для проверки обученных моделей на адекватность, сравним полученные значения метрик accuracy моделей с baseline, которым будет выступать среднее значение ("доля тарифов") рассчитанная по группам по имеющимся данным.

In [123]:
# выделение пяти групп по имеющимся признакам
df['calls_group'] = pd.qcut(df['calls'], 5)
df['minutes_group'] = pd.qcut(df['minutes'], 5)
df['messages_group'] = pd.qcut(df['messages'], 5)
df['mb_used_group'] = pd.qcut(df['mb_used'], 5)

# получение скора модели, как доли тарифов по группам признаков
avg_model = (
    df.groupby(['calls_group', 'minutes_group', 'messages_group', 'mb_used_group'])
    ['is_ultra'].mean()
    .reset_index()
)
# обогащение скором данным
df['score'] = avg_model['is_ultra']

# объем выборки небольшой, некоторый скор не рассчитался, 
# так как достигнута уникальность сочетания четырех признаков,
# для которых в выборке нет информаци о применяемом тарифе 
# для полученых таким образом возможных клиентов
df['score'] = df['score'].fillna(df['score'].mean())

# определение целевой перемнной по значению скора
df.loc[df['score'] > 0.5, 'avg_model_prediction'] = 1
df.loc[df['score'] <= 0.5, 'avg_model_prediction'] = 0

# расчет показателя доли верно предсказанных тарифов
print('Модель "человеческого обучения"')
print('Accuracy:', accuracy_score(df['is_ultra'], df['avg_model_prediction']))

Модель "человеческого обучения"
Accuracy: 0.6876166770379589


Таким образом, предсказания модели, полученой путем группировки признаков, имеют долю верных предстказаний 0,6876, что значительно ниже всех изученных в ходе эксперимента моделей.

## Выводы

На исследование была предсталена выборка клиентов мобильного оператора, имеющая четыре признака и одну целевую перемнную - тарифный план. Требовалось построить модели распознания клиента.

В ходе эксперимента были построены модели логистической регрессии, дерева решений и случайного леса. Метрикой успеха эксперимента была определена accuracy - доля верный предсказаний модели.

Личший результат - 0,8238 продемонсрировала модель случайного леса с количеством лесов - 30 и количеством слоев деревьев - 9. Посредсвенный результат оказался у логистической регресии - 0,7398.

Адекватность эксперементальных моделей проверялась автором исследования путем сравнения с моделью, полученной полученой путем группировки признаков, которая имеюе долю верных предсказаний 0,6876.

Таким образом, по показателю accuracy лучшей моделью для предсказания тарифного плана клиента по имеющимся данным стоит признать RandomForestClassifier(max_depth=9, n_estimators=30).

Модель на тестовой выборке показала результат метрики accuracy равный 0.8780. Метрика превысила уровень в 0,7500 - заявленный стейкхолдером. Эксперимет стоит признать успешным.